In [1]:
import numpy as np
import pandas as pd
import os
import glob
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 500
np.random.seed(231)
from sklearn.model_selection import train_test_split
import cv2


In [2]:
train_dir = "/mnt/disks/large/data/original_train/"
data_dir = "/mnt/disks/large/data/"
output_dir = "/home/rugezhao/CS231N_CZT/data/"

In [3]:
train_df = pd.read_csv(data_dir+"train.csv", engine='python')
labels_df = pd.read_csv(data_dir+"labels.csv", engine='python')


The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2. To stratify sample the class needs at least 3 samples. So I decided to drop all samples that are tagged to a label that appeared less than 3 times.

# Data Split

In [8]:
def train_val_test_split(train_id_attribute_id, ratios = (0.8,0.1,0.1),min_count=3):
    train_id_attribute_id_ohe= np.zeros((len(train_df), len(labels_df)), dtype=int)
    for idx, attr_arr in enumerate(train_df.attribute_ids.str.split(" ").apply(lambda l: list(map(int, l))).values):
        train_id_attribute_id_ohe[idx, attr_arr] = 1
    train_id_attribute_id_ohe = pd.DataFrame(train_id_attribute_id_ohe,columns=labels_df.attribute_id)
    print("one hot encoding done")
    
    col_sum=pd.DataFrame(np.sum(train_id_attribute_id_ohe,axis=0))
    col_sum=col_sum.reset_index()
    infreq_labels = list(col_sum[col_sum.iloc[:,1] < min_count].iloc[:,0])
    print("{} infrequent labels less than {} times found".format(len(infreq_labels),min_count))
    
    # drop the examples with infreq labels, i.e. if 1 in any of these columns -->drop
    df_onehot = pd.concat([train_id_attribute_id,train_id_attribute_id_ohe], axis=1)
    df_onehot = df_onehot.drop(columns=["attribute_ids"])
    infreq_cols = df_onehot[infreq_labels]
    X_infreq = list(df_onehot.loc[(infreq_cols!=0).any(axis=1)].id)
    
    df_onehot = df_onehot[~df_onehot.id.isin(X_infreq)]
     
    X = df_onehot.id
    y = df_onehot.iloc[:,1:]
    # use stratify=y for stratified sampling -->> impossible!! since they need to treat combination of label as a class
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratios[2], random_state=231,stratify=None)
    print("test set done")
    # use stratify =y_train for stratified sampling -->impossible!!
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=ratios[1], random_state=231,stratify=None)
    print("val set done")
    print("train set done")
    return X_train, X_val, X_test, X_infreq, df_onehot

In [9]:
np.random.seed(231)
X_train, X_val, X_test, X_infreq, df_onehot = train_val_test_split(train_df)

one hot encoding done
26 infrequent labels less than 3 times found
test set done
val set done
train set done


## Check distribution

In [10]:
df_one_train = df_onehot[df_onehot.id.isin(X_train)]
df_one_val = df_onehot[df_onehot.id.isin(X_val)]
df_one_test = df_onehot[df_onehot.id.isin(X_test)]

In [11]:
col_pct_train=pd.DataFrame(np.sum(df_one_train.iloc[:,1:],axis=0))/df_one_train.shape[0]
col_pct_val=pd.DataFrame(np.sum(df_one_val.iloc[:,1:],axis=0))/df_one_val.shape[0]
col_pct_test=pd.DataFrame(np.sum(df_one_test.iloc[:,1:],axis=0))/df_one_test.shape[0]

In [12]:
col_pct_train.sort_values(0,ascending=False)

,0
813,0.183829
1092,0.130805
147,0.124440
189,0.094695
13,0.083141
671,0.077681
51,0.070298
194,0.068195
1059,0.059818
121,0.059501


In [13]:
col_pct_val.sort_values(0,ascending=False)

,0
813,0.185083
1092,0.128205
147,0.119760
189,0.092796
13,0.088523
671,0.075092
51,0.067766
194,0.063797
1059,0.059320
121,0.058506


In [14]:
col_pct_test.sort_values(0,ascending=False)

,0
813,0.172711
1092,0.132418
147,0.122253
189,0.099451
13,0.084890
671,0.073626
194,0.067308
51,0.066575
121,0.064469
1059,0.062912


In [15]:
# export split to csv
train_df['train'] = 0
train_df['val']=0
train_df['test']=0


In [16]:
train_df.loc[train_df.id.isin(list(X_train)),'train'] = 1
train_df.loc[train_df.id.isin(list(X_val)),'val'] = 1
train_df.loc[train_df.id.isin(list(X_test)),'test'] = 1
train_df.to_csv(output_dir+"/train_split.csv",index=False)

# TODO: Change Folder Structure - move .png according to train_split.csv

In [17]:
# TODO
# import os
# train_files = os.listdir(train_dir)

,id,attribute_ids,train,val,test
0,1000483014d91860,147 616 813,1,0,0
1,1000fe2e667721fe,51 616 734 813,0,0,1
2,1001614cb89646ee,776,1,0,0
3,10041eb49b297c08,51 671 698 813 1092,1,0,0
4,100501c227f8beea,13 404 492 903 1093,1,0,0


# Training preprocessing

In [ ]:
# pause for now since it can be done at run time using pytorch
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Val, Test Preprocessing

In [ ]:
# pause for now since it can be done at run time using pytorch
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Training Per Channel Mean and Std - after preprocessing

In [22]:
# import os
# train_files = os.listdir(train_dir)
# do not use folder structure, use the newly created training data

train_files=list(train_df[train_df.train==1].id)
train_files = [x+".png" for x in train_files]

In [23]:
sum_x_rgb = np.array([0.0]*3)
sum_x2_rgb = np.array([0.0]*3)
file_count =0
pixel_count=0
for i, file in enumerate(train_files):
    if i%10==0:
        print("{}/{} files read".format(i+1, len(train_files)))
    file_count+=1
    
    image = cv2.imread(train_dir+"/"+file)
    pixel_count += image.shape[0]*image.shape[1]
    # R, G, B
    for i in range(3):
        sum_x_rgb[i] += np.sum(image[:,:,i].astype(float))
        sum_x2_rgb[i] += np.sum(image[:,:,i].astype(float)**2)
        
        

1/88452 files read
2/88452 files read
3/88452 files read
4/88452 files read
5/88452 files read
6/88452 files read
7/88452 files read
8/88452 files read
9/88452 files read
10/88452 files read
11/88452 files read
12/88452 files read
13/88452 files read
14/88452 files read
15/88452 files read
16/88452 files read
17/88452 files read
18/88452 files read
19/88452 files read
20/88452 files read
21/88452 files read
22/88452 files read
23/88452 files read
24/88452 files read
25/88452 files read
26/88452 files read
27/88452 files read
28/88452 files read
29/88452 files read
30/88452 files read
31/88452 files read
32/88452 files read
33/88452 files read
34/88452 files read
35/88452 files read
36/88452 files read
37/88452 files read
38/88452 files read
39/88452 files read
40/88452 files read
41/88452 files read
42/88452 files read
43/88452 files read
44/88452 files read
45/88452 files read
46/88452 files read
47/88452 files read
48/88452 files read
49/88452 files read
50/88452 files read
51/88452 

406/88452 files read
407/88452 files read
408/88452 files read
409/88452 files read
410/88452 files read
411/88452 files read
412/88452 files read
413/88452 files read
414/88452 files read
415/88452 files read
416/88452 files read
417/88452 files read
418/88452 files read
419/88452 files read
420/88452 files read
421/88452 files read
422/88452 files read
423/88452 files read
424/88452 files read
425/88452 files read
426/88452 files read
427/88452 files read
428/88452 files read
429/88452 files read
430/88452 files read
431/88452 files read
432/88452 files read
433/88452 files read
434/88452 files read
435/88452 files read
436/88452 files read
437/88452 files read
438/88452 files read
439/88452 files read
440/88452 files read
441/88452 files read
442/88452 files read
443/88452 files read
444/88452 files read
445/88452 files read
446/88452 files read
447/88452 files read
448/88452 files read
449/88452 files read
450/88452 files read
451/88452 files read
452/88452 files read
453/88452 fil

798/88452 files read
799/88452 files read
800/88452 files read
801/88452 files read
802/88452 files read
803/88452 files read
804/88452 files read
805/88452 files read
806/88452 files read
807/88452 files read
808/88452 files read
809/88452 files read
810/88452 files read
811/88452 files read
812/88452 files read
813/88452 files read
814/88452 files read
815/88452 files read
816/88452 files read
817/88452 files read
818/88452 files read
819/88452 files read
820/88452 files read
821/88452 files read
822/88452 files read
823/88452 files read
824/88452 files read
825/88452 files read
826/88452 files read
827/88452 files read
828/88452 files read
829/88452 files read
830/88452 files read
831/88452 files read
832/88452 files read
833/88452 files read
834/88452 files read
835/88452 files read
836/88452 files read
837/88452 files read
838/88452 files read
839/88452 files read
840/88452 files read
841/88452 files read
842/88452 files read
843/88452 files read
844/88452 files read
845/88452 fil

1198/88452 files read
1199/88452 files read
1200/88452 files read
1201/88452 files read
1202/88452 files read
1203/88452 files read
1204/88452 files read
1205/88452 files read
1206/88452 files read
1207/88452 files read
1208/88452 files read
1209/88452 files read
1210/88452 files read
1211/88452 files read
1212/88452 files read
1213/88452 files read
1214/88452 files read
1215/88452 files read
1216/88452 files read
1217/88452 files read
1218/88452 files read
1219/88452 files read
1220/88452 files read
1221/88452 files read
1222/88452 files read
1223/88452 files read
1224/88452 files read
1225/88452 files read
1226/88452 files read
1227/88452 files read
1228/88452 files read
1229/88452 files read
1230/88452 files read
1231/88452 files read
1232/88452 files read
1233/88452 files read
1234/88452 files read
1235/88452 files read
1236/88452 files read
1237/88452 files read
1238/88452 files read
1239/88452 files read
1240/88452 files read
1241/88452 files read
1242/88452 files read
1243/88452

1588/88452 files read
1589/88452 files read
1590/88452 files read
1591/88452 files read
1592/88452 files read
1593/88452 files read
1594/88452 files read
1595/88452 files read
1596/88452 files read
1597/88452 files read
1598/88452 files read
1599/88452 files read
1600/88452 files read
1601/88452 files read
1602/88452 files read
1603/88452 files read
1604/88452 files read
1605/88452 files read
1606/88452 files read
1607/88452 files read
1608/88452 files read
1609/88452 files read
1610/88452 files read
1611/88452 files read
1612/88452 files read
1613/88452 files read
1614/88452 files read
1615/88452 files read
1616/88452 files read
1617/88452 files read
1618/88452 files read
1619/88452 files read
1620/88452 files read
1621/88452 files read
1622/88452 files read
1623/88452 files read
1624/88452 files read
1625/88452 files read
1626/88452 files read
1627/88452 files read
1628/88452 files read
1629/88452 files read
1630/88452 files read
1631/88452 files read
1632/88452 files read
1633/88452

...

86750/88452 files read
86751/88452 files read
86752/88452 files read
86753/88452 files read
86754/88452 files read
86755/88452 files read
86756/88452 files read
86757/88452 files read
86758/88452 files read
86759/88452 files read
86760/88452 files read
86761/88452 files read
86762/88452 files read
86763/88452 files read
86764/88452 files read
86765/88452 files read
86766/88452 files read
86767/88452 files read
86768/88452 files read
86769/88452 files read
86770/88452 files read
86771/88452 files read
86772/88452 files read
86773/88452 files read
86774/88452 files read
86775/88452 files read
86776/88452 files read
86777/88452 files read
86778/88452 files read
86779/88452 files read
86780/88452 files read
86781/88452 files read
86782/88452 files read
86783/88452 files read
86784/88452 files read
86785/88452 files read
86786/88452 files read
86787/88452 files read
86788/88452 files read
86789/88452 files read
86790/88452 files read
86791/88452 files read
86792/88452 files read
86793/88452

87116/88452 files read
87117/88452 files read
87118/88452 files read
87119/88452 files read
87120/88452 files read
87121/88452 files read
87122/88452 files read
87123/88452 files read
87124/88452 files read
87125/88452 files read
87126/88452 files read
87127/88452 files read
87128/88452 files read
87129/88452 files read
87130/88452 files read
87131/88452 files read
87132/88452 files read
87133/88452 files read
87134/88452 files read
87135/88452 files read
87136/88452 files read
87137/88452 files read
87138/88452 files read
87139/88452 files read
87140/88452 files read
87141/88452 files read
87142/88452 files read
87143/88452 files read
87144/88452 files read
87145/88452 files read
87146/88452 files read
87147/88452 files read
87148/88452 files read
87149/88452 files read
87150/88452 files read
87151/88452 files read
87152/88452 files read
87153/88452 files read
87154/88452 files read
87155/88452 files read
87156/88452 files read
87157/88452 files read
87158/88452 files read
87159/88452

87500/88452 files read
87501/88452 files read
87502/88452 files read
87503/88452 files read
87504/88452 files read
87505/88452 files read
87506/88452 files read
87507/88452 files read
87508/88452 files read
87509/88452 files read
87510/88452 files read
87511/88452 files read
87512/88452 files read
87513/88452 files read
87514/88452 files read
87515/88452 files read
87516/88452 files read
87517/88452 files read
87518/88452 files read
87519/88452 files read
87520/88452 files read
87521/88452 files read
87522/88452 files read
87523/88452 files read
87524/88452 files read
87525/88452 files read
87526/88452 files read
87527/88452 files read
87528/88452 files read
87529/88452 files read
87530/88452 files read
87531/88452 files read
87532/88452 files read
87533/88452 files read
87534/88452 files read
87535/88452 files read
87536/88452 files read
87537/88452 files read
87538/88452 files read
87539/88452 files read
87540/88452 files read
87541/88452 files read
87542/88452 files read
87543/88452

87860/88452 files read
87861/88452 files read
87862/88452 files read
87863/88452 files read
87864/88452 files read
87865/88452 files read
87866/88452 files read
87867/88452 files read
87868/88452 files read
87869/88452 files read
87870/88452 files read
87871/88452 files read
87872/88452 files read
87873/88452 files read
87874/88452 files read
87875/88452 files read
87876/88452 files read
87877/88452 files read
87878/88452 files read
87879/88452 files read
87880/88452 files read
87881/88452 files read
87882/88452 files read
87883/88452 files read
87884/88452 files read
87885/88452 files read
87886/88452 files read
87887/88452 files read
87888/88452 files read
87889/88452 files read
87890/88452 files read
87891/88452 files read
87892/88452 files read
87893/88452 files read
87894/88452 files read
87895/88452 files read
87896/88452 files read
87897/88452 files read
87898/88452 files read
87899/88452 files read
87900/88452 files read
87901/88452 files read
87902/88452 files read
87903/88452

88227/88452 files read
88228/88452 files read
88229/88452 files read
88230/88452 files read
88231/88452 files read
88232/88452 files read
88233/88452 files read
88234/88452 files read
88235/88452 files read
88236/88452 files read
88237/88452 files read
88238/88452 files read
88239/88452 files read
88240/88452 files read
88241/88452 files read
88242/88452 files read
88243/88452 files read
88244/88452 files read
88245/88452 files read
88246/88452 files read
88247/88452 files read
88248/88452 files read
88249/88452 files read
88250/88452 files read
88251/88452 files read
88252/88452 files read
88253/88452 files read
88254/88452 files read
88255/88452 files read
88256/88452 files read
88257/88452 files read
88258/88452 files read
88259/88452 files read
88260/88452 files read
88261/88452 files read
88262/88452 files read
88263/88452 files read
88264/88452 files read
88265/88452 files read
88266/88452 files read
88267/88452 files read
88268/88452 files read
88269/88452 files read
88270/88452

In [25]:
channel_mean = sum_x_rgb/pixel_count
channel_mean

array([144.26212231, 155.88824173, 165.01235545])

In [26]:
channel_sd = np.sqrt(sum_x2_rgb/pixel_count-channel_mean**2)
channel_sd

array([64.59720922, 63.78302281, 63.95205758])

# TODO:  Generate small training sample for debugging 

sample by id from large training sample

In [ ]:
sample_idx = np.random.choice(train_df.shape[0], 5000)
sample_train_df = train_df.iloc[sample_idx,:]

In [24]:
#  TODO: to folder structure and move those files to a separate folder

1